# CartPole Gym environment with TfAgents

## Install packages (gym, tfagents, tensorflow,....)

In [1]:
!pip install easyagents >/dev/null

##### If in colab: load additional packages for rendering

In [2]:
import sys

if 'google.colab' in sys.modules:
    !apt-get install xvfb >/dev/null
    !pip install pyvirtualdisplay >/dev/null    
    
    from pyvirtualdisplay import Display
    Display(visible=0, size=(960, 720)).start()    

## Dry Run (very short training, no logging)

In [3]:
from easyagents.tfagents import PpoAgent
from easyagents.config import TrainingDurationFast
from easyagents.config import LoggingSilent

ppoAgent = PpoAgent( gym_env_name='CartPole-v0', training_duration=TrainingDurationFast(), logging=LoggingSilent())
ppoAgent.train()

### Visualize Training (average returns, losses)

In [4]:
ppoAgent.plot_average_rewards()

In [5]:
ppoAgent.plot_losses()

## Default training with movie  (self-defined network, default logging)

In [6]:
from easyagents.tfagents import PpoAgent

In [7]:
ppoAgent = PpoAgent( gym_env_name = 'CartPole-v0', fc_layers=(100, 50, 25) )
ppoAgent.train()

### Visualize Training (average returns, losses)

In [8]:
ppoAgent.plot_average_rewards()

In [9]:
ppoAgent.plot_losses()

### Visualize Trained Policy (playing a game)

In [10]:
from IPython.display import HTML

HTML( ppoAgent.render_episodes_to_html(num_episodes=1, fps=50) )

## Next: Custom training (self-defined training scheme & logging)

* see [Berater on colab](https://colab.research.google.com/github/christianhidber/easyagents/blob/master/jupyter_notebooks/easyagents_berater.ipynb)
  (an example of a gym environment implementation based on a routing problem)

